In [1]:
# The objective of this project is to build a scraper able to retrieve review/ rating data from video-game websites
# I used a Kaggle dataset as base for this project: https://www.kaggle.com/rush4ratio/video-game-sales-with-ratings
# This dataset contains games with sales data and some ratings,and I added information to it

import requests
import fnmatch
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup

In [2]:
# Main Extraction Functions

# go to a webpage and return a list with the content extracted by beautifull soup
def get_content(url):
    # Most websites refuse GET requests from python, so we change the header to pretend we're a browser.
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.76 Safari/537.36'}
    page = requests.get(url, headers = headers)
    soup = BeautifulSoup(page.content, 'html.parser')
    return list(soup.children)

# Search a list and return the first matching item index
# used to search a list of 'children' and return the index of which lists contain the searched value
def get_index(myList, search):
    for idx, item in enumerate(myList):
        tags = str(myList[idx])
        if fnmatch.fnmatch(tags, search):
            return idx

# find the children containing the searched value and get it's children
# keeps going until the defined limit of children is reached

# ToDo: sometimes we reach a list with only one children, but inside that children theres more than one 
# the lookup can't really reach it's final objective and needs a 'little push'
# this means I have to extract it's children and then use the function again... need to find an easier way
def lookup(myList, desiredSize, search): 
    while len(myList) > desiredSize:
        # sometimes the searched term is in the tag itself, so we can't see it once we get a list of children
        # to avoid that I used a try/catch that breaks the loop and return the previous list
        try:
            soup = myList[get_index(myList, search)]
            myList = list(soup.children)
        except:
            break
    
    return myList

In [3]:
# cleaning functions

def clean_text(raw_text, char_list):
    clean_text = str(raw_text)
    for c in char_list :
        clean_text = clean_text.replace(c, '')
    return  clean_text.lower()   

def clean_list(df, score, title, year):
    char_list = ['.', ':', '-', '!', '/', '\\']
    remove_list = []

    for idx, e in enumerate(df['Name']):
        remove = False
        
        if df.loc[idx, score] != None:
            og_name = clean_text(e, char_list)
            scrap_name = clean_text(df.loc[idx, title],char_list)
            if og_name != scrap_name:
                if df.loc[idx, 'Year_of_Release'] != df.loc[idx, year]:
                    remove = True
        else:
            remove = True
        if remove:
            remove_list.append(idx)
    
    return remove_list

In [4]:
# For GameSpot

# assemble search queries with the game titles from the csv
def get_gs_sources(dataFrame):
    url = []
    query = 'https://www.gamespot.com/search/?q='
    title = ''
    counter = 0

    for game in df['Name']:
        myList = get_content('https://www.gamespot.com/search/?q=' + game.replace(' ','+'))
        
        search = '*editorial river search-results*'
        temp = lookup(myList, 1, search)
        temp = str(temp).split('>')[3]
        url.append('https://www.gamespot.com' + temp.split('"')[1])
        
        # progress track
        counter += 1
        if counter % 10 == 0:
            print(counter)
    
    return url

# get title, year, review score, metacritic score, and user score from gs
def scrap_gs(urls):
    counter = 0
    games = []
    rel_years = []
    r_scores = []
    m_scores = []
    u_scores = []
    
    for url in urls:
        myList = get_content(url)
        
        #Get Title
        search = '*title*'
        temp = lookup(myList, 2, search)

        title = str(temp).split("'")[1]
        title = title.replace(' - GameSpot','')
        games.append(title)
        
        # Get Release Year
        search = '*First Released*'
        temp = lookup(myList, 2, search)

        if (temp != myList):
            temp = str(temp[1]).split(', ')

            temp = clean_text(temp, ['<span>','</span>'])
            #releaseY = str(releaseY).replace('<span>','')
            #releaseY = releaseY.replace('</span>','')           
            rel_years.append(temp)
        else:
            rel_years.append(None)        

        # Get Review Score
        search = '*Review score*'
        temp = lookup(myList, 1, search)

        if (temp != myList):
            temp = str(temp).split('>')
            temp = temp[3].split('<')
            r_scores.append(temp[0])
        else:
            r_scores.append(None)

        # Get Metacritic Score
        search = '*Metascore*'
        temp = lookup(myList, 2, search)

        if (temp != myList):
            temp = str(temp[0]).split('blank">')
            temp = temp[1].split('<')
            m_scores.append(temp[0])
        else:
            m_scores.append(None)

        # Get User Score
        search = '*userAvg*'
        temp = lookup(myList, 2, search)

        if (temp != myList):
            temp = str(temp).split('\n')
            temp = temp[2].replace(' ','')
            u_scores.append(temp)
        else:
            u_scores.append(None)    
        
        # progress track
        counter += 1
        if counter % 10 == 0:
            print(counter)  
            
    return games, rel_years, r_scores, m_scores, u_scores

In [5]:
# For IGN

# assemble search query, get content of result and save the url to a list
def get_ign_sources(df):
    source = []
    counter = 0
    for name in df['Name']:
        url = 'https://ca.ign.com/search?q=' + name.replace(' ','%20')
        myList = get_content(url)

        # Get source
        search = '*search-item-title*'
        temp = lookup(myList, 2, search)[1]
        temp = str(temp).split('"')[1]
        source.append(temp)
        
        # progress track
        counter += 1
        if counter % 10 == 0:
            print(counter)
        
    return source

# get title, year and review score of ign
def scrap_ign(urls):
    counter = 0
    game = []
    year = []
    score = []
    
    for url in urls:
        myList = get_content(url)

        # Get title
        search = '*title*'
        temp = lookup(myList, 1, search)
        if (temp != myList):
            temp = str(temp).split(' - ')[0][2:]
            game.append(str(temp))
        else:
            game.append(None)

        # Get score
        search = '*ratingValue*'
        temp = lookup(myList, 1, search)
        if (temp != myList):
            temp = str(temp).split(',')
            if len(temp) > 4:            
                temp = temp[-3].split(':')[1]
            else:
                temp = str(temp)[9:20].replace(' ','')
            score.append(str(temp))
        else:
            score.append(None)

        # Get year
        search = '*Released</dt><dd class=*'
        temp = lookup(myList, 1, search)
        if (temp != myList):
            # give it a push and search again
            temp = lookup(list(temp[0].children), 1, search)
            temp = str(temp).split(',')[2]
            temp = temp.split('<')[0].replace(' ','')
            year.append(temp)
        else:
            year.append(None)

        # progress track, to keep ansiety level
        counter += 1
        if counter % 10 == 0:
            print(counter)
            
    return game, year, score

In [6]:
print('reading game titles from source')
df = pd.read_csv('Video_Games_Sales_2016.csv')

print('\n getting urls for scraping IGN:')
url = get_ign_sources(df)

print('\n collecting contents from IGN:')
g, y, s = scrap_ign(url)

reading game titles from source

 getting urls for scraping IGN:
10
20
30
40
50
60
70
80
90

 collecting contents from IGN:
10
20
30
40
50
60
70
80
90


In [7]:
df['IGN_title'] = g
df['IGN_year'] = y
df['IGN_score'] = s
df['IGN_source'] = url

df.head()

,Name,Platform,Year_of_Release,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,Critic_Score,Critic_Count,User_Score,User_Count,Developer,Rating,IGN_title,IGN_year,IGN_score,IGN_source
0,Wii Sports,Wii,2006,Sports,Nintendo,41.36,28.96,3.77,8.45,82.53,76.0,51.0,8.0,322.0,Nintendo,E,Wii Sports,2006,7.5,https://ca.ign.com/games/wii-sports/wii-826987
1,Super Mario Bros.,NES,1985,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24,NaN,NaN,NaN,NaN,NaN,NaN,Super Mario Bros. 2,1988,8.5,https://ca.ign.com/games/super-mario-bros-2/ne...
2,Mario Kart Wii,Wii,2008,Racing,Nintendo,15.68,12.76,3.79,3.29,35.52,82.0,73.0,8.3,709.0,Nintendo,E,Mario Kart Wii,2008,8.9,https://ca.ign.com/games/mario-kart-wii/wii-94...
3,Wii Sports Resort,Wii,2009,Sports,Nintendo,15.61,10.93,3.28,2.95,32.77,80.0,73.0,8.0,192.0,Nintendo,E,Wii Sports Resort Video Review,None,None,https://ca.ign.com/videos/2009/07/16/wii-sport...
4,Pokemon Red/Pokemon Blue,GB,1996,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37,NaN,NaN,NaN,NaN,NaN,NaN,Pokemon,2000,None,https://ca.ign.com/tv/pokemon


In [8]:
print('\n getting urls for scraping gs:')
url = get_gs_sources(df)

print('\n collecting contents from gs:')
g, y, r, m, u = scrap_gs(url)


 getting urls for scraping gs:
10
20
30
40
50
60
70
80
90

 collecting contents from gs:
10
20
30
40
50
60
70
80
90


In [9]:
df['GS_title'] = g
df['GS_year'] = y
df['GS_review'] = r
df['GS_metacritic'] = m
df['GS_user'] = u
df['GS_source'] = url

df.head()

,Name,Platform,Year_of_Release,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,...,IGN_title,IGN_year,IGN_score,IGN_source,GS_title,GS_year,GS_review,GS_metacritic,GS_user,GS_source
0,Wii Sports,Wii,2006,Sports,Nintendo,41.36,28.96,3.77,8.45,82.53,...,Wii Sports,2006,7.5,https://ca.ign.com/games/wii-sports/wii-826987,Wii Sports,"['nov 19', '2006']",7.8,76,8.1,https://www.gamespot.com/wii-sports/
1,Super Mario Bros.,NES,1985,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24,...,Super Mario Bros. 2,1988,8.5,https://ca.ign.com/games/super-mario-bros-2/ne...,Super Mario Bros.,"['oct 18', '1985']",6.8,None,8.9,https://www.gamespot.com/classic-nes-series-su...
2,Mario Kart Wii,Wii,2008,Racing,Nintendo,15.68,12.76,3.79,3.29,35.52,...,Mario Kart Wii,2008,8.9,https://ca.ign.com/games/mario-kart-wii/wii-94...,Mario Kart Wii,"['apr 27', '2008']",8.5,82,8.7,https://www.gamespot.com/mario-kart-wii/
3,Wii Sports Resort,Wii,2009,Sports,Nintendo,15.61,10.93,3.28,2.95,32.77,...,Wii Sports Resort Video Review,None,None,https://ca.ign.com/videos/2009/07/16/wii-sport...,Wii Sports Resort,"['jul 26', '2009']",8,80,8.3,https://www.gamespot.com/wii-sports-resort/
4,Pokemon Red/Pokemon Blue,GB,1996,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37,...,Pokemon,2000,None,https://ca.ign.com/tv/pokemon,Pokemon GO,"['jul 6', '2016']",7,69,6.1,https://www.gamespot.com/pokemon-go/


In [10]:
# ToDo: add this to a function

# adjust gs date
for i, date in enumerate(df['GS_year']):
    test = date
    test = clean_text(test, ["[", "]", "'"])
    test = test.split(' ')
    df.loc[i, 'GS_year'] = test[-1]

df['GS_year'].head()

0    2006
1    1985
2    2008
3    2009
4    2016
Name: GS_year, dtype: object

In [11]:
# Clean IGN
remove_list = clean_list(df, 'IGN_score', 'IGN_title', 'IGN_year')

print(len(remove_list), ' rows of IGN data removed')

for i in remove_list:
    df.loc[i, 'IGN_title'] = None
    df.loc[i, 'IGN_year'] = None
    df.loc[i, 'IGN_score'] = None
    df.loc[i, 'IGN_source'] = None

# Clean GS
remove_list = clean_list(df, 'GS_user', 'GS_title', 'GS_year')

print(len(remove_list), ' rows of GS data removed')

for i in remove_list:
    df.loc[i, 'GS_title'] = None
    df.loc[i, 'GS_year'] = None
    df.loc[i, 'GS_review'] = None
    df.loc[i, 'GS_user'] = None
    df.loc[i, 'GS_metacritic'] = None
    df.loc[i, 'GS_source'] = None

48  rows of IGN data removed
19  rows of GS data removed


In [12]:
df.to_csv('new.csv',  index=False)